In [1]:
import os
import pandas as pd

# Set the base directory
base_dir = "/mnt/disk15tb/mmpatil/MEA_Analysis_New/StimulationAnalysis/new/"

# List to store dataframes
df_list = []

# Traverse the directory structure
for root, dirs, files in os.walk(base_dir):
    for file in files:
        if file == "network_summary_metrics.csv":
            file_path = os.path.join(root, file)
            try:
                # Read the CSV file, skip the first row and set the second row as header
                df = pd.read_csv(file_path, skiprows=1)
                df_list.append(df)
            except Exception as e:
                print(f"Error reading {file_path}: {e}")

# Combine all dataframes
if df_list:
    combined_df = pd.concat(df_list, ignore_index=True)
        # Reorder columns
    columns_order = [
        "Date", "Wellplate ID", "Well Number", "Assay Run ID", 
        "Well Group Name", "Well Group Color", "Unnamed: 51", 
        "Control", "Assay Tag", "DIV"
    ]
    

    # Ensure all columns are included (both in specified order and the rest)
    reordered_columns = columns_order + [col for col in combined_df.columns if col not in columns_order]

    combined_df = combined_df[reordered_columns]
    # Group by 'runID' and perform any aggregation if needed
    combined_df = combined_df.sort_values('Assay Run ID') # Example: calculating mean for each group

    # Save the grouped dataframe to a new CSV
    output_path = f"{base_dir}/combined_grouped_metrics.csv"
    combined_df.to_csv(output_path)
    print(f"Combined and grouped data saved to {output_path}")
else:
    print("No CSV files found.")

Combined and grouped data saved to /mnt/disk15tb/mmpatil/MEA_Analysis_New/StimulationAnalysis/new//combined_grouped_metrics.csv


In [3]:
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# Pre-coded control and stimulated well numbers
control_wells = [2,4,6]
stimulated_wells =  [1, 3,5]

# Filter data for control and stimulated wells
control_data = combined_df[combined_df['Well Number'].isin(control_wells)]
stimulated_data = combined_df[combined_df['Well Number'].isin(stimulated_wells)]


In [4]:
print(control_data)

          Date Wellplate ID  Well Number  Assay Run ID Well Group Name   
47  2024-12-27       M06792            6            89          WT_60K  \
43  2024-12-27       M06792            2            89         HET_80K   
45  2024-12-27       M06792            4            89         WT_100K   
51  2024-12-27       M06792            4           106         WT_100K   
49  2024-12-27       M06792            2           106         HET_80K   
53  2024-12-27       M06792            6           106          WT_60K   
55  2024-12-27       M06792            2           108         HET_80K   
57  2024-12-27       M06792            4           108         WT_100K   
59  2024-12-27       M06792            6           108          WT_60K   
25  2024-12-27       M06792            2           110         HET_80K   
29  2024-12-27       M06792            6           110          WT_60K   
27  2024-12-27       M06792            4           110         WT_100K   
67  2024-12-27       M06792           

In [5]:
combined_df['Assay Run ID'].unique()

array([ 89, 106, 108, 110, 113, 115, 117, 120, 122, 124, 127, 129, 131,
       134, 136, 138])

In [6]:
# Dictionary to rename Assay Run IDs
# Baseline ID
baseline_id = 89

# rename_assay_run_ids = {
#     64: "Baseline",
#     67: "Post Stimulation 1",
#     69: "Post Stimulation 2"
  
# }
rename_assay_run_ids = {
    89: "Baseline",
    106: "Post Stimulation 10min",
    108: "Post Stimulation 20min",
    110: "Post Stimulation 30min",
    113: "Post Stimulation 10 min",
    115: "Post Stimulation 20 min",
    117: "Post Stimulation 30 min",
    120: "Post Stimulation 10 min",
    122: "Post Stimulation 20 min",
    124: "Post Stimulation 30 min",
    127: "Post Stimulation 10 min",
    129: "Post Stimulation 20 min",
    131: "Post Stimulation 30 min",
    134: "Post Stimulation 10 min",
    136: "Post Stimulation 20 min",
    138: "Post Stimulation 30 min"


}

# Extract metrics that have "Mean" in the column name
metrics = [col for col in combined_df.columns if "Mean" in col]
metrics = ['Burst Frequency'] + metrics  # Add 'Burst Frequency' to metrics



# Create a PDF to save all plots
pdf_filename = "stimulation_plots_with_percentage_change_Dec28.pdf"
with PdfPages(pdf_filename) as pdf:
    for metric in metrics:
        # === Original Plot: Absolute Values ===
        plt.figure(figsize=(14, 8))

        # Extract unique run IDs and sort them for consistent plotting
        run_ids = sorted(combined_df['Assay Run ID'].unique())

        # Bar width
        bar_width = 0.15
        x_positions = range(len(run_ids))  # X-axis positions for Run IDs

        # Offset for each well within a run ID group
        offsets = {
            "control": [i * bar_width for i in range(len(control_wells))],
            "stimulated": [i * bar_width for i in range(len(stimulated_wells))]
        }

        # Plot control wells grouped by run IDs
        for idx, well in enumerate(control_wells):
            well_data = control_data[control_data['Well Number'] == well]
            well_means = [well_data[well_data['Assay Run ID'] == run][metric].mean() for run in run_ids]
            plt.bar(
                x=[x + offsets['control'][idx] for x in x_positions],
                height=well_means,
                label=f"Control Well {well}",
                alpha=0.7,
                width=bar_width
            )

        # Plot stimulated wells grouped by run IDs
        for idx, well in enumerate(stimulated_wells):
            well_data = stimulated_data[stimulated_data['Well Number'] == well]
            well_means = [well_data[well_data['Assay Run ID'] == run][metric].mean() for run in run_ids]
            plt.bar(
                x=[x + offsets['stimulated'][idx] + len(control_wells) * bar_width for x in x_positions],
                height=well_means,
                label=f"Stimulated Well {well}",
                alpha=0.7,
                width=bar_width
            )

        # Rename the Assay Run IDs for x-axis labels
        renamed_run_ids = [rename_assay_run_ids.get(run, run) for run in run_ids]

        # Add titles and labels
        plt.title(f"{metric} Comparison Across Run IDs", fontsize=16)
        plt.xlabel("Assay Run IDs", fontsize=14)
        plt.ylabel(metric, fontsize=14)
        plt.xticks(
            [x + (len(control_wells) + len(stimulated_wells)) * bar_width / 2 for x in x_positions],
            renamed_run_ids,
            rotation=45, ha="right", fontsize=10
        )
        plt.legend(fontsize=10)
        plt.tight_layout()

        # Save the plot to the PDF
        pdf.savefig()
        plt.close()

        # === Additional Plot: Percentage Change from Baseline ===
        plt.figure(figsize=(14, 8))

        # Calculate percentage change from baseline
        baseline_means = combined_df[combined_df['Assay Run ID'] == baseline_id].set_index('Well Number')[metric]
        for idx, well in enumerate(control_wells + stimulated_wells):
            well_data = combined_df[combined_df['Well Number'] == well]
            pct_change = [
                ((well_data[well_data['Assay Run ID'] == run][metric].mean() - baseline_means.get(well, 0)) / baseline_means.get(well, 1e-6)) * 100
                for run in run_ids
            ]
            offset_group = "control" if well in control_wells else "stimulated"
            offset_idx = idx % len(control_wells + stimulated_wells)  # Unique offset for each bar
            plt.bar(
                x=[x + offset_idx * bar_width for x in x_positions],
                height=pct_change,
                label=f"{'Control' if well in control_wells else 'Stimulated'} Well {well}",
                alpha=0.7,
                width=bar_width
            )

        # Add titles and labels
        plt.title(f"{metric} Percentage Change from Baseline Across Run IDs", fontsize=16)
        plt.xlabel("Assay Run IDs", fontsize=14)
        plt.ylabel(f"Percentage Change in {metric}", fontsize=14)
        plt.xticks(
            [x + (len(control_wells) + len(stimulated_wells)) * bar_width / 2 for x in x_positions],
            renamed_run_ids,
            rotation=45, ha="right", fontsize=10
        )
        plt.legend(fontsize=10)
        plt.tight_layout()

        # Save the plot to the PDF
        pdf.savefig()
        plt.close()

print(f"All plots grouped by renamed Run IDs saved to {pdf_filename}")

All plots grouped by renamed Run IDs saved to stimulation_plots_with_percentage_change_Dec28.pdf


In [14]:
combined_df.columns


Index(['Date', 'Wellplate ID', 'Well Number', 'Assay Run ID',
       'Well Group Name', 'Well Group Color', 'Unnamed: 51', 'Control',
       'Assay Tag', 'DIV', 'Instance', 'Burst Frequency',
       'Spikes within Bursts', 'Mean Spikes per Burst', 'Spikes per Burst std',
       'Spikes per Burst CV', 'Median Spikes per Burst',
       'Spikes per Burst 10th percentile', 'Spikes per Burst 90th percentile',
       'Mean Spikes per Burst per Electrode',
       'Spikes per Burst per Electrode std',
       'Spikes per Burst per Electrode CV',
       'Median Spikes per Burst per Electrode',
       'Spikes per Burst per Electrode 10th percentile',
       'Spikes per Burst per Electrode 90th percentile', 'Mean Burst Duration',
       'Burst Duration std', 'Burst Duration CV', 'Median Burst Duration',
       'Burst Duration 10th percentile', 'Burst Duration 90th percentile',
       'Mean Burst Peak Firing Rate', 'Burst Peak Firing Rate std',
       'Burst Peak Firing Rate CV', 'Median Burst Peak